In [10]:
from requests_oauthlib import OAuth1Session
from requests.exceptions import ConnectionError, ReadTimeout, SSLError
from elasticsearch import Elasticsearch
import json, datetime, time, pytz, re, sys,traceback, pymongo
from pymongo import MongoClient
from collections import defaultdict
import numpy as np

In [11]:
KEYS = { # 自分のアカウントで入手したキーを下記に記載
        'consumer_key':'en35U31mW3P9EAqZvR6KXkZ2h',
        'consumer_secret':'JWoHyJU5KRnbeWjb17hVHAQsiPFgwGwvBkZSkRsARrApZqNEKd',
        'access_token':'868744152753647617-ei1ptGYVwJqgv2zy7f09fPVuFrsYbLY',
        'access_secret':'VHQQ1g1MDTUMRufrF2mojkxiig47qEc7ghcGj32fyMX1f',
       }
twitter = None
connect = None
db      = None
tweetdata = None
meta    = None

In [12]:
def initialize(): # twitter接続情報や、mongoDBへの接続処理等initial処理実行
    global twitter, twitter, connect, db, tweetdata, meta
    twitter = OAuth1Session(KEYS['consumer_key'],KEYS['consumer_secret'],
                            KEYS['access_token'],KEYS['access_secret'])
#   connect = Connection('localhost', 27017)     # Connection classは廃止されたのでMongoClientに変更 
    connect = MongoClient('localhost', 27017)
    db = connect.trend
    tweetdata = db.recipe_tweetdata
    meta = db.recipe_metadata

In [13]:
# 文字列を日本時間2タイムゾーンを合わせた日付型で返す
def str_to_date_jp(str_date):
    dts = datetime.datetime.strptime(str_date,'%a %b %d %H:%M:%S +0000 %Y')
    return pytz.utc.localize(dts).astimezone(pytz.timezone('Asia/Tokyo'))

# 現在時刻をUNIX Timeで返す
def now_unix_time():
    return time.mktime(datetime.datetime.now().timetuple())
    
# 日付の文字列をDatetime型で返す
def str_to_date_jp(str_date):
    dts = datetime.datetime.strptime(str_date,'%a %b %d %H:%M:%S +0000 %Y')
    return pytz.utc.localize(dts).astimezone(pytz.timezone('Asia/Tokyo'))

# UTCの日付文字列を日本時間にしてDatetime型で返す
def utc_str_to_jp_str(str_date):
    dts = datetime.datetime.strptime(str_date,'%a %b %d %H:%M:%S +0000 %Y')
    return pytz.utc.localize(dts).astimezone(pytz.timezone('Asia/Tokyo')).strftime("%Y/%m/%d %H:%M:%S")


def str_to_date(str_date):
    dts = datetime.datetime.strptime(str_date,'%Y-%m-%d %H:%M:%S')
    return pytz.utc.localize(dts)

def str_to_date_jp_utc(str_date):
    return datetime.datetime.strptime(str_date,'%Y-%m-%d %H:%M:%S') - datetime.timedelta(hours=9)

def date_to_Japan_time(dts):
    return pytz.utc.localize(dts).astimezone(pytz.timezone('Asia/Tokyo'))

def date_to_Japan_time_str(dts):
    return pytz.utc.localize(dts).astimezone(pytz.timezone('Asia/Tokyo')).strftime("%Y/%m/%d %H:%M:%S")

def date_to_str(dt):
    return dt.strftime("%Y/%m/%d %H:%M:%S")

def str_to_unix_date_jp(str_date):
    dts = datetime.datetime.strptime(str_date,'%a %b %d %H:%M:%S +0000 %Y')
    dt = pytz.utc.localize(dts).astimezone(pytz.timezone('Asia/Tokyo'))
    return time.mktime(dt.timetuple())

def unix_time_to_datetime(int_date):
    return datetime.datetime.fromtimestamp(int_date)

In [90]:
def select_outlier_retweet_num_per_hour(from_str_datetime_jp, limit=120):
    '''
    from_str_datetime_jp: １時間枠の開始時刻
    limit: この数を超えてリツイートされたものを検出する
    e.g. select_outlier_tweet_num_per_hour("2015-03-18 22:00:00")
    '''
    result_list = []
    from_date = str_to_date_jp_utc(from_str_datetime_jp)
    to_date = str_to_date_jp_utc(from_str_datetime_jp) + datetime.timedelta(hours=1) 

    for d in tweetdata.find({'retweeted_status':{"$ne": None},'created_datetime':{"$gte":from_date, "$lt":to_date}},\
                            {'user':1,'text':1,'entities':1, 'created_at':1, 'id':1}):
        mensioned_username = ""
        if len(d['entities']['user_mentions'])!=0:
            mensioned_username = d['entities']['user_mentions'][0]['screen_name']

        result_list.append({"created_at":utc_str_to_jp_str(d['created_at']),\
                            "screen_name":d['user']['screen_name'],\
                            "referred_name":mensioned_username,\
                            "text":d['text'].replace('\n',' ')\
                            })

    name_dict = defaultdict(int)
    for r in result_list:
        name_dict[r['referred_name']] += 1  

    s = sorted(name_dict.items(),key=lambda x: x[1],reverse=True) # リツイート回数でソート
    return s[0:int(np.sum(list(map(lambda x: 1 if x[1]>limit else 0 ,s))))] # リツイート元ユーザー名, リツイート回数(limitを超えたもの)

In [98]:
start_date = str_to_date_jp_utc("2017-06-23 11:00:00")
to_date    = str_to_date_jp_utc("2017-06-23 13:00:00")
d_diff = (to_date - start_date)
d_hours = d_diff.days * 24 + d_diff.seconds/float(3600)

for i in range(int(d_hours)):
    d = (start_date + datetime.timedelta(hours=i)).strftime("%Y-%m-%d %H:%M:%S")
    result = select_outlier_retweet_num_per_hour(d, limit=1)
    if len(result) > 0:
        print (d, result)

2017-06-23 02:00:00 [('okumuratorucc', 29), ('ApAopoo', 17), ('okayado1215', 8), ('CChannel_tv', 7), ('gomio123', 6), ('map_u_chin', 6), ('colabo_yumeno', 5), ('yako_FLPR3', 4), ('yolu831', 4), ('akita11', 3), ('Shinsuke1', 3), ('usa_sable', 3), ('jupi420gab', 3), ('sorasorayaada', 3), ('shunichitakano', 3), ('skchammm', 3), ('110Hongo', 3), ('yukakuramoti', 3), ('onoderamasaru', 2), ('euro_tour', 2), ('Hortensia_PR', 2), ('2chourou', 2), ('onshokukiko', 2), ('odencentury', 2), ('LecoleVantan2T', 2), ('etwaaru', 2), ('aichi_34', 2), ('uswow02', 2), ('costmary_net', 2), ('cvk_kd', 2), ('moa__tsukino', 2), ('matsujun5213', 2), ('aZOf0woXmJCSz6J', 2), ('Potetosalad73', 2), ('uni_mafumafu', 2), ('nao_kakaomame', 2), ('sukisukiaraisan', 2), ('jaja_uma_', 2), ('blendykotya', 2), ('910123mai2', 2), ('the_Vambi', 2), ('J_League', 2), ('eda_neko', 2), ('SasimiAlexander', 2), ('mahya99', 2), ('iseebitarou_com', 2), ('toshi_seapara', 2)]
2017-06-23 03:00:00 [('onoda_kimi', 16), ('okumuratorucc', 

1

In [94]:
s = sorted(name_dict.items(),key=lambda x:x[1],reverse=True) # リツイート回数でソート
t=s[0:int(np.sum(list(map(lambda x: 1 if x[1]>2 else 0 ,s))))] # リツイート元ユーザー名, リツイート回数(limitを超えたもの)

In [95]:
t

[('okumuratorucc', 29),
 ('ApAopoo', 17),
 ('okayado1215', 8),
 ('CChannel_tv', 7),
 ('gomio123', 6),
 ('map_u_chin', 6),
 ('colabo_yumeno', 5),
 ('yako_FLPR3', 4),
 ('yolu831', 4),
 ('akita11', 3),
 ('Shinsuke1', 3),
 ('usa_sable', 3),
 ('jupi420gab', 3),
 ('sorasorayaada', 3),
 ('shunichitakano', 3),
 ('skchammm', 3),
 ('110Hongo', 3),
 ('yukakuramoti', 3)]

In [61]:
key(name_dict)

0

In [69]:
fruit = ['strawberry','grape','orange','strawberry','grape']
d = defaultdict(int)
for f in fruit:
    d[f] += 1 #デフォルト値がintで初期化されているのでインクリメントできる
print(d)
print (sorted(d, key = lambda x:x[1], reverse=True))

defaultdict(<class 'int'>, {'strawberry': 2, 'grape': 2, 'orange': 1})
['strawberry', 'grape', 'orange']


In [74]:
s = sorted(name_dict.items(),key=lambda x:x[1],reverse=True) # リツイート回数でソート
s

[('okumuratorucc', 29),
 ('ApAopoo', 17),
 ('okayado1215', 8),
 ('CChannel_tv', 7),
 ('gomio123', 6),
 ('map_u_chin', 6),
 ('colabo_yumeno', 5),
 ('yako_FLPR3', 4),
 ('yolu831', 4),
 ('akita11', 3),
 ('Shinsuke1', 3),
 ('usa_sable', 3),
 ('jupi420gab', 3),
 ('sorasorayaada', 3),
 ('shunichitakano', 3),
 ('skchammm', 3),
 ('110Hongo', 3),
 ('yukakuramoti', 3),
 ('onoderamasaru', 2),
 ('euro_tour', 2),
 ('Hortensia_PR', 2),
 ('2chourou', 2),
 ('onshokukiko', 2),
 ('odencentury', 2),
 ('LecoleVantan2T', 2),
 ('etwaaru', 2),
 ('aichi_34', 2),
 ('uswow02', 2),
 ('costmary_net', 2),
 ('cvk_kd', 2),
 ('moa__tsukino', 2),
 ('matsujun5213', 2),
 ('aZOf0woXmJCSz6J', 2),
 ('Potetosalad73', 2),
 ('uni_mafumafu', 2),
 ('nao_kakaomame', 2),
 ('sukisukiaraisan', 2),
 ('jaja_uma_', 2),
 ('blendykotya', 2),
 ('910123mai2', 2),
 ('the_Vambi', 2),
 ('J_League', 2),
 ('eda_neko', 2),
 ('SasimiAlexander', 2),
 ('mahya99', 2),
 ('iseebitarou_com', 2),
 ('toshi_seapara', 2),
 ('oja_map', 1),
 ('yuzpon5', 1),